# Introduction to Data Science and Systems 2020-2021<small><small>v20202021a</small></small>
## Lab 4: Data Science in practice
#### - ***you should submit this notebook on Moodle along with one pdf file (see the end of the notebook and Moodle for instructions)***
---
#### University of Glasgow


$$
\newcommand{\vec}[1]{ {\bf #1}} 
\newcommand{\real}{\mathbb{R}}
\DeclareMathOperator*{\argmin}{arg\,min}
\newcommand{\expect}[1]{\mathbb{E}[#1]}
$$

## Purpose of this lab

In the lab you will apply some of the data science and systems techniques you have learned about previously. The aim is to use the basic techniques to solve a specific problems related to accessing and storing data in a Panda dataframe. 

#### Part 1:
* revisit how to use Pandas to easily load and inspect a dataset 

#### Part 2:
* implement a solution for compressing the values in the dataset based on PCA

####  Part 3:
* investigate the relationship between query cardinality and the query itself
* sample a set of realistic queries using your knowledge of probability density functions
* collect a dataset of queries and cardinality using Pandas 
* suggest and implement your own solution for predicting the cardinality of the query (using linear algebra)



***Notice:*** 
- This lab is more open-ended than the previous labs and there are fewer step-by-step instructions.
- It is a brand new lab so there are bound to be a few glitches, so don't hesitate to ask if you get stuck.


---

# Before you begin

Please update the tools we use for the automated grading by running the below command (uncomment) and restart your kernel (and then uncomment again) -- or simply perform the installation externally in an Anaconda/Python prompt.

In [1]:
#!pip install -U --force-reinstall --no-cache https://github.com/johnhw/jhwutils/zipball/master

In [ ]:
# Standard imports
# Make sure you run this cell!
from __future__ import print_function, division
import numpy as np  # NumPy
import scipy.stats 
import os
import pandas as pd
import sys
import timeit
import time
import binascii
from unittest.mock import patch
from uuid import getnode as get_mac

from jhwutils.checkarr import array_hash, check_hash, check_scalar, check_string
import jhwutils.image_audio as ia
import jhwutils.tick as tick

###
tick.reset_marks()

# special hash funciton
def case_crc(s, verbose=True):
    h_crc = binascii.crc32(bytes(s.lower(), 'ascii'))
    if verbose:
        print(h_crc)
    return h_crc

# this command generaties a unique key for your system/computer/account
uuid_simple = (("%s") % get_mac())
uuid_str = ("%s\n%s\n%s\n%s\n%s\n") % (os.path,sys.path,sys.version,sys.version_info,get_mac())
uuid_system = case_crc(uuid_str,verbose=False) 


# Set up Matplotlib
import matplotlib as mpl   
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('figure', figsize=(8.0, 4.0), dpi=140)
np.random.seed(2019)

# You can ignore this
print("np",np.__version__)
print("pd",pd.__version__)
print("Everything imported OK (ignore deprecated warnings)")

In [ ]:
# Hidden cell for utils needed when grading (you can/should not edit this)

---

**Mini-task**: provide your personal details in two variables:

* `student_id` : a string containing your student id (e.g. "1234567x"), must be 8 chars long.
* `student_typewritten_signature`: a string with your name (e.g. "Adam Smith") which serves as a declaration that this is your own work (read the declaration of originality when you submit on Moodle).

In [ ]:
student_id = "" # your 8 char student id
student_typewritten_signature = "" # your full name, avoid spceical chars if possible

# YOUR CODE HERE
raise NotImplementedError()

---

## Part 1 Loading and inspecting the data
The system admins are planning on storing the data in a Panda dataframe ("the database") insted of a more traditional database.
However, they are concerned about the performance of the Panda-based system and have decided to investigate further.

#### Part 1.1 Download the dataset
Download the Covertype Data Set from the UCI repository and save it in a subfolder called data (i.e. when loading you should load from "./data/covtype.data")

https://archive.ics.uci.edu/ml/datasets/Covertype

#### Part 1.2 Loading the dataset
Load the dataset in covtype.data into a Panda dataframe and add appropriate headers to the first 11 columns (at least) according to the dataset description (i.e. the first column should be "Elevation"). Hint: consider using the "rename" method.


In [ ]:
data = pd.read_csv("./data/covtype.data", header=None) # DO NOT CHANGE THIS!

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Check that you have (most likely) the right data set and have added headers.
with tick.marks(6):        
    assert(np.sum(data['Elevation'].to_numpy()) == 1719426752)    
    for i in range(0,11):
        assert(isinstance(data.columns[i],str))
        

#### Part 1.3 Basic stats
We can use Panda's `describe` function to extract some useful statistics

In [ ]:
data.describe()

From the table above, identify the median and mean for the `Elevation` measurements and save the values in variables named `stats_mean_elevation` and `stats_median_elevation` (two decimals). 

***Hint***: You may need to do a bit of investigation into what the 'describe' function outputs.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with tick.marks(4):    
    assert(check_hash(stats_mean_elevation, ((), 14796.849999999999)))
    

In [ ]:
with tick.marks(4):    
    assert(check_hash(stats_median_elevation, ((), 14980.0)))
    

----

### Part 2 Dataset Compression 

#### Part 2.1
In an attempt to optimise the database the team is looking to use PCA to compress the dataset (i.e. reduce the number of bits we need to store).

It is speculated that the following measurements can be represented in a low dimensional space without too much loss of information.
>    - 'Elevation'
>    - 'Aspect'
>    - 'Slope'
>    - 'Horizontal_Distance_To_Hydrology'
>    - 'Vertical_Distance_To_Hydrology'
>    - 'Horizontal_Distance_To_Roadways'
>    - 'Hillshade_9am'
>    - 'Hillshade_Noon '
>    - 'Hillshade_3pm '
>    - 'Horizontal_Distance_To_Fire_Points '
>    - 'Wilderness_Area[0]  '

1) Extract the relevant data in a separate numpy array (do not modify the dataframe itself). Demean and standardize the extracted data such that each column has zero mean and a standard deviation of one.

2) Compute the PCA (use np.linalg.eig) and determine the minimum number of components required to preserve 70% of variance. Save the number of components in `n_components`. The relative contribution from one principle component is measured as $\frac{\lambda_i}{\sum_{j=1}^J \lambda_j}$ where $\lambda$ is the eigenvalue for that component and $J$ is the number of eigenvalues.  

3) Provide the actual "compressed" coordinates in `data_lowd` with dimension (581012, n_components).


***Note:*** There are no visible tests in this part as we expect you will be able to solve this given the knowledge you have acquired in earlier labs.


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
### Hidden test checking n_components [10 marks]


In [ ]:
### Hidden test test checking data_lowd of shape (581012, n_components) [15 marks]

---

## Part 3: Cardinality of the result set

#### Part 3.1 Queries
The database team is interested in optimising the performance of the system when querying the dataframe. To test the performance, queries are assumed to relate only to the Elevation column and are specified as an interval. For example, a query [2500,2061] would return all entries where the Elevation is between 2500 and 2061.

The actual query which is executed by the Panda is then specified as $[a,b]$ or $[c-s/2, c+s/2]$. A probability density has been formulated which is believed to generate a set of realistic queries.

$$p(c) = 0.3\cdot Normal(c|2500,50) + 0.4\cdot Normal(c|3000,100) + 0.3\cdot Normal(c|3250,80)\,\,\,\,\,\,\,\,\, (Eq. 1)$$  

It is made up of three socalled components with each component being a Normal distribution.

The s (the size of the internal) is sampled according.

$$p(s) = Gamma(s|8,25)\,\,\,\,\,\,\,\,\, (Eq. 2)$$

#### Part 3.2 Sample a set of queries
You must now complete a function which can sample 2000 queries from the query distribution defined by the equations in Eq. 1 and Eq. 2.

Save the queries $[c,s]$ as rows in `X` (i.e. `X` should be of size (2000,2) ).

***Hint***: For $p(s)$, first sample a component with the given probability, then sample from that component. Remember to truncate the c values below zero!


In [ ]:
def sample_queries(N=2000):    
        
    # YOUR CODE HERE
    raise NotImplementedError()
    return X

Let's sample...

In [ ]:
np.random.seed(1111) # you may wan tto provide a seed to consistent results
X = sample_queries(N=2000)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1,1,1)
ax.hist(X[:,0])
ax.hist(X[:,1])
ax.legend(('c','s'))


In [ ]:
# Note this test only ensure you you have sensible sampler
# - it is not absolute evidence 
# that your sampler is exactly as specified (you should make 
# extra sure to test your implementtaion)
#
# You sampler will be checked manually when marking against a more 
# extentive test and you marks a thus tentative!
#

with tick.marks(10):        
    query_function_is_likely_correct = False
    for irep in range(0,3): # repeat a few times to make it wasn't a lucky run
        for n in [100000]:
            samples = sample_queries(n)
            assert samples.shape == (n,2)
            assert np.min(samples[:,0]) > 0
            assert np.min(samples[:,1]) > 0            
            qs  = np.quantile(samples[:,0], np.linspace(0.1,0.95,10))
            qc  = np.quantile(samples[:,1], np.linspace(0.1,0.95,10))                                                    
            qs_true = np.array([2478.04573054, 2518.19126684, 2588.47755615, 2917.62945017,
            2984.9975373 , 3045.53633682, 3117.29174297, 3193.0221899,
            3255.41404674, 3327.7111032])            
            qc_true =  np.array([116.55924418, 138.49041163, 155.92921611, 171.96901016,
            187.79187985, 204.68888341, 223.16652347, 245.08779129,
            275.02758806, 328.7319762])            
            assert( np.all( np.abs(qs_true - qs) < 25 ))
            assert( np.all( np.abs(qc_true - qc) < 10 ))                
    
    query_function_is_likely_correct = True
    

In [ ]:
# A placeholder for the automarker

#### Part 3.3 Execute a single query 
You must now complete a function which can execute a single query based on the start and end points of the interval, [a,b], against the database (in this case a Panda dataframe stored in the variable named ´data´)

The function must return:
- the time taken in seconds (already provided), t
- the cardinality of the result set, n
- the result set, i.e. all rows where $a=c-s/2 < Elevation < b=c+s/2$

***Hint***: You may need to consult the documentation for Panda to find a suitable command.

In [ ]:
def query(a,b):    
    tic=timeit.default_timer()    
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    toc=timeit.default_timer() 
    t = toc - tic 
    return t, n, res          

In [ ]:
with tick.marks(5): 
    query_function_is_correct = False
    t, n, res = query(2967.0,2967.5)
    z = np.sum(res['Aspect'].to_numpy())    
    assert( n == 762 )
    assert( z == 121183 )
    query_function_is_correct = True
    

#### Part 3.4 Execute and time all queries

You should now execute all the queries in `X` against the database by calling the function `query`. Queries should be read from the `X` array previously generated.

The resulting data should be collected in two numpy arrays:

- `y_times` with shape (2.000,1) holding the times measured for each query
- `y_cardinality` with shape (2.000,1) holding the size of the result set for each query


In [ ]:
N = np.shape(X)[0]
y_times = np.zeros((N,1))
y_cardinality = np.zeros((N,1))

# Hint remember that you need to parse a and b to query (not c and s)
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Hidden test (make sure you test your function carefully!) [5 marks]


We can then plot the distribution of the cardinality measurements... Validate that the histogram looks like you expect.


In [ ]:
fig = plt.figure(figsize=(4,4)) 
ax = fig.add_subplot(1,1,1) 
ax.hist(y_cardinality)
ax.set_xlabel('Value')
ax.set_ylabel('Count')
ax.set_title('Orginal')

#### Part 3.5 Save the data for later use (and for testing/validation )

In [ ]:
np.savez('datalog.npz', X=X,y_times=y_times,y_cardinality=y_cardinality)

---

#### Part 3.6 Map from query to cardinality
We are now interested in specifying a function which can map from a query (i.e. x=[c,s]) to the query time for this database (i.e the Panda dataframe).

The function is pre-defined as:: 

$$f(x;\theta ) = {w_0} + {w_1}c + {w_2}s + {w_3}{c^2} + {w_4}{s^2} + {w_5}{c^3} + {w_6}{s^3} + {w_7}{c^4} + {w_8}{s^4} + {w_9}\cdot c \cdot s \,\,\,\,\,\,\,\, (Eq. 3)$$

where x is a query, y is the query time in seconds. The loss defined as 

$$L\left( \theta  \right) = \frac{1}{N} \sum\limits_{n = 1}^{N=2000} {{{\left( { {y_n} - f({x_n};\theta )} \right)}^2}} $$

You should now complete four classes/methods that the database team use in their estimation:

- `output process` 
- `input process`
- `predict`
- `estimate`
- `evaluate`

**Hint**: It is recommend that you write the function in matrix form (use pen and paper) before attempting to implement it.

Any method you implement must be based on pure Python and numpy (i.e. **you cannot use scikit learn or similar tools**).


**Marking:** The overall marking is based on whether you manage to a) beat a baseline prediction, and b) come close to the correct solution. There are intermediate marks to help you along.



In [ ]:
# Load the generated data from the file
# This makes it easier to mark and debug your solution at marking time
# Hint: If you struggle to generate meaningful data in the previous part you can 
# copy the content of the cell to a new cell and load the file called datalog_demo.npz 
# which contains pre-generated observations.

tmp = np.load('datalog.npz')
X = tmp['X']
y_cardinality = tmp['y_cardinality']

In [ ]:
# A placeholder for the automarker

#### Part 3.7 Input and output processing
Complete the `input_process` class for preprocessing the inputs, X (e.g. normalisaiton and other transforms of X and y).

- `output_process`: is already provided and you shouldn't need to modify it. 
- `input_process`: you probably need to inspect the other functions and cells below (e.g. predict) to work out what to put here.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
class input_process:   
    """    
    A simple class which contains two functions for processign the 2D observation of the query x=(c,s) 
    and returns a potentailly scaled, normalised and expanded representation.                         
    """
    X_mean = None # proposal; the usage depends on you decicion to normalise X or not    
    X_std  = None # proposal; the usage depends on you decicion to standardise X or not    
        
    def __init__(self, X):         
        self.X_mean = 0 # this is a trick, it is only 0 for now
        self.X_std  = 1 # this is a trick, it is only 1 for now
        self.X_mean = np.mean(self.process(X)[:,1:] ,axis=0)                
        self.X_std  = np.std(self.process(X)[:,1:] ,axis=0)                
                    
    def process(self,X):
        """
        A function which takes the 2D in and processes the data, e.g. make a basis expansion and 
        standaise the result.
        
        """
        # YOUR CODE HERE
        raise NotImplementedError()
        return X

##### Input process:

We can apply the input process to query and observe the effect. This is the tricky bit so make sure to validate that input_process works as intended (there are no automated test to check this!).

**Note**: If you have done this the recommend/intended way the output would not be 2D for each observations but rather 10D.

In [ ]:
inprocess = input_process(X)
x_demo = inprocess.process(X[0:3,:]) # take 3 queries and see what happends to them
print(x_demo)

#### Part 3.8 Predict 
Implement a function that can predict the cardinality, $y$, of a query $x=[c,s]$ based on Eq. (3). The parameters are stored in `theta` (i.e. $w \in \mathbb{R}^{10}$), using only linear algebra/vector operations and calls to the relevant input and output process functions (these are shown). 

The function should return the estimate of the cardinality in $y$, and the scale prediction in y_prime. It must be able to make predictions for multiple observations in X (i.e. X is a matrix).

In [ ]:
def predict(X, theta, output_process,input_process):      
    if input_process is not None: # note we use None when testing/making
        X = input_process.process(X)
    
    # Hint: Only a single line of code is missing
    # YOUR CODE HERE
    raise NotImplementedError()
    
    if output_process is not None: 
        y = output_process.rescale(y_prime)
    else:
        y = y_prime
        
    return y, y_prime

In [ ]:
# Hidden test validating the predict function [5 Marks]
# Test which test the core functionality of the predict function (it dones't test the input and output process)

with tick.marks(5):
    X_test = np.array([[1.2,8.54],[2.4,4.5]])
    theta = np.array([0.50722768, -1.32649421])
    y_test, y_prime_test = predict(X_test, theta, None, None)
        
    assert(check_hash(y_test, ((2,), -48.182417152)))
    assert(check_hash(y_prime_test, ((2,), -48.182417152)))    
    

#### Part 3.9
Write a function which computes the mean squared error (mse) based on the difference between the predicted and true query cardinality. It should also return the mean absolute error and the releative absolute error (in pct) wrt to the actual observation for the absolute error (i.e. how many pct is the prediction wrong on average).




In [ ]:
def evaluate(y_pred,y_true):
    # YOUR CODE HERE
    raise NotImplementedError()
    return error_mse, error_abs, error_abs_relative

In [ ]:
# Test checkign the evaluate function[TODO MARKS]
with tick.marks(5):
    a,b,c = evaluate(np.array([4.5,6.2,-100.10]), np.array([14.5,-26.2,-110.10]))    
    assert(check_hash(a, ((), 6248.8)))
    assert(check_hash(b, ((), 87.33333333333333)))
    assert(check_hash(c, ((), -106.30209505074754)))      

#### Part 3.10
Implement a function which estimates the parameters, i.e. the $w$'s stored in `theta`, of the function based on the squared error loss defined above.

Hint: This should be done using only matrix operations and calls to the input/output process functions. You don't need numerical optimisation for this.


In [ ]:
def estimate(X, y, outprocess, inprocess):
    """
    X: 2D queries/observations 
    y: observed times (the orginal ones, i.e. not scaled or normalised) 
    outprocess: an instance of the output process class which is used to scale and rescale y
    inprocess: an instance of the input process class which is used to process the 2D input in X in a sensible way
    """        
    if outprocess is not None: # note we use None when testing/making
        y = outprocess.scale(y)
        
    if inprocess is not None: # note we use None when testing/making
        X = inprocess.process(X)
    
    # Hint: 2-3 lines of code missing
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return theta

In [ ]:
# Hidden test validating the estimate function[5 MARKS]
with tick.marks(7):
    X_test = np.array([[1.2,8.54,9.77],[2.4,4.5,-33.2]])
    y_yest = np.array([1,2])    
    theta_test = estimate(X_test, y_yest, None, None)
    assert(check_hash(theta_test, ((3,), 0.5095082848719855)))


#### Part 3.11

We can now put it all together and check if we are able to make sensible fit to the data.

Note: It might be necessary to revisit your implementation above in case you do not get a sensible fit in the first attempt.


In [ ]:
inprocess = input_process(X)
outprocess = output_process(y_cardinality)
theta = estimate(X,y_cardinality,outprocess,inprocess)
y_pred, y_prime_pred = predict(X, theta, outprocess,inprocess)
error_mse, error_abs, error_abs_rel = evaluate(y_pred,y_cardinality)

print("The mse error is: ", error_mse)
print("The abs error is: ", error_abs)
print("The releative abs error is [pct]: ", error_abs_rel)

#### Part 3.12

To evaluate the quality of your fit it is often useful to produce visualisation to aid our analysis. For this purpose you should now:

- Create a plot of the cardinality vs the ones estimated by the model. Include a line to illustrate the ideal prediction.
- Create a figure showing the distribution of errors
- Create a figure visualising the squared error against the the query time 


Note: These are not assessed per se but suggested for your benefit. They will be written to the pdf file and inspected manually.

In [ ]:
fig_312a = plt.figure(figsize=(5,5)) # DO NOT CHANGE
ax_312a = fig_312a.add_subplot() # DO NOT CHANGE - YOU MUST ADD YOUR FIGURE TO THIS AXIS
#######################################################
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
fig_312b = plt.figure(figsize=(5,5)) # DO NOT CHANGE
ax_312b = fig_312b.add_subplot(1,1,1) # DO NOT CHANGE - YOU MUST ADD YOUR FIGURE TO THIS AXIS
#######################################################
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
fig_312c = plt.figure(figsize=(5,5)) # DO NOT CHANGE
ax_312c = fig_312c.add_subplot(1,1,1) # DO NOT CHANGE - YOU MUST ADD YOUR FIGURE TO THIS AXIS
#######################################################

# YOUR CODE HERE
raise NotImplementedError()

#### Part 3.13 Baselines
To determine if your model is worth deploying and using for estimating the database load, your database team has requested you compare your function/fit to a naiive baseline.


The two baseline are:
- a) use the mean of the observation as the prediction for all queries 
- b) use the median of the observed query times as the prediction for all queries.   

Complete the `predict_average` and `predict_median` function below and compare with your predictive model.


In [ ]:
def predict_average(y):    
    # YOUR CODE HERE
    raise NotImplementedError()
    return y_avg

In [ ]:
def predict_median(y):    
    # YOUR CODE HERE
    raise NotImplementedError()
    return y_med

In [ ]:
# Check that the baselines work as expected
with tick.marks(4):
    baseline_is_corrrect= False
    y_test = np.array([-3.4, 3, 6.2, 10])
    y_test_avg = predict_average(y_test)    
    y_test_median = predict_median(y_test)
    assert(check_hash(y_test_avg, ((), 19.75)))
    assert(check_hash(y_test_median, ((), 23.0)))
    baseline_is_corrrect = True
    

In [ ]:
# Evaluate the baselines

y_pred_bl = predict_average(y_cardinality)
error_l2_bl, error_abs_bl, error_abs_rel_bl = evaluate(y_pred_bl,y_cardinality)
print("The mse error is: ", error_l2_bl)
print("The abs error is: ", error_abs_bl)
print("The relative abs error in the orginal domain is [pct]: ", error_abs_rel_bl)

y_pred_blmed = predict_median(y_cardinality)
error_l2_bl, error_abs_bl, error_abs_rel_bl = evaluate(y_pred_blmed,y_cardinality)
print("The mse error is: ", error_l2_bl)
print("The abs error is: ", error_abs_bl)
print("The relative abs error is [pct]: ", error_abs_rel_bl)


Overall marking of your function...

In [ ]:
# Hidden test, you get 5 marks for beating the median baseline (baseline needs to be correct for any marks)

In [ ]:
## Hidden test, you get 20 marks for getting for a competetive performance close to our implementation 
# You'd need to do you own validation that this is a sensible and competetive solution.



#### No more assessed questions... remember to generate the pdf!


### Part 3.X: Optional/challenges
Several improvements could be considered for the function/implementation in Part 3:
- The function we have specified in Eq. (3) is unlikely to be the best option for this purpose. You may want to try the neural network we implemented in Lab 2 - or perhaps other variations of the function suggested. 
- The cardinality is an integer, but our prediction function does not take that into account. 
- We handle the negative predictions by truncating the predictions; ideally, this should be inherent to the model (i.e., via the loss function).
- The function we have fitted probably only works well for the specific query dataset; we should test it on an unseen query dataset - or at least held-out-queries from the current dataset - to make sure it generalizes.
- Can you also predict the query time using this approach?
- ...

We would be happy to look over notebooks addressing these issues, but you will not get extra credit.

-----

# Submission on Moodle


We will generate the **one** pdf file you'll need to submit along with the notebook.

*Note*: you do not need to worry about the formatting etc (that's predetermined)


In [ ]:
## Report generation - YOU MUST YOU RUN THIS CELL !
#
# Ignore warnings regarding fonts
#
from matplotlib.backends.backend_pdf import PdfPages

# Declaration of originality with system info
try:
    f = open('uofg_declaration_of_originality.txt','r')
    uofg_declaration_of_originality = f.read()
except: 
    uofg_declaration_of_originality = "uofg_declaration_of_originality not present in cwd"

try:
    student_id.lower()
except: 
    student_id="NORESPONSE"
try:
    student_typewritten_signature.lower()
except: 
    student_typewritten_signature="NORESPONSE"

fn = ("idss_lab_04_dsinpractice_%s_declaration.pdf" % (student_id.lower()))
fig_dec = plt.figure(figsize=(10, 12)) 
fig_dec.text(0.1,0.1,("%s\n\n Student Id %s\n\n Typewritten signature: %s\n\n UUID System: %s" % (uofg_declaration_of_originality,student_id, student_typewritten_signature, uuid_system)))
    
# Combined: 
fn = ("idss_lab_04_dsinpractice_%s_combined_v20202021a.pdf" % (student_id))
pp = PdfPages(fn)
pp.savefig(fig_312a)
pp.savefig(fig_312b)
pp.savefig(fig_312c)
pp.savefig(fig_dec)
pp.close()

with tick.marks(0):  # have you generated the combined file...? you don't actually get any credit for this; just confirmation that the file has been generated
    assert(os.path.exists(fn))

**You must (for full or partial marks) submit TWO files via Moodle:**

- this notebook (completed) after "Restart and rerun all":
    - `idss_lab_04_dsinpractice_v20202021a.ipynb`
    
- the combined pdf (autogenerated) containing the relevant figures and answers for the manual marking:
     - `idss_lab_04_dsinpractice_[YOUR STUDENT ID]_combined_v20202021a.pdf`)     
    


---

# Appendix: Marking Summary (and other metadata)
#### - make sure the notebook runs without errors (remove/resolve the `raise NotImplementedError()`) and "Restart and Rerun All" cells to get a correct indication of your marks.

In [ ]:
print("Marks total : ","100")
print("Marks visible (with immediate feedback): ","50")
print("Marks hidden (without immediate feedback): ","50")
print("\nThe fraction below displays your performance on the autograded part of the lab that's visible with feedback (only valid after `Restart and Run all`:")
tick.summarise_marks() # 

---